# Simulador de colas en cine

### Complementos necesarios

In [2]:
#!pip install simpy

### Librerias

In [1]:
import simpy
import random
import pandas as pd

#### Creacion de clientes

In [2]:

class Persona:
    ultimo_id = 0
        
    def __init__(self):
        Persona.ultimo_id += 1
        self.id = Persona.ultimo_id
        self.edad = random.randint(14, 60)
        self.genero = random.choice(["Femenino", "Masculino"])
        self.nivel_paciencia = random.randint(1, 3)
        
    def tiempo_decidir(self):
        return self.edad * 0.02 + self.nivel_paciencia

#### Definir numero de personas

In [8]:
num_personas = 40

#### Generacion de n clientes

In [9]:

personas = [Persona() for _ in range(num_personas)]

# Crear DataFrame
df = pd.DataFrame(
    data={
        "Identificacion": [persona.id for persona in personas],
        "Edad": [persona.edad for persona in personas],
        "Genero": [persona.genero for persona in personas],
        "Nivel_Paciencia": [persona.nivel_paciencia for persona in personas],
        "Tiempo_Decidir (min)": [persona.tiempo_decidir() for persona in personas]
    }
)

In [10]:
df

,Identificacion,Edad,Genero,Nivel_Paciencia,Tiempo_Decidir (min)
0,51,42,Masculino,2,2.84
1,52,50,Femenino,1,2.00
2,53,50,Femenino,2,3.00
3,54,47,Femenino,3,3.94
4,55,59,Masculino,2,3.18
5,56,24,Masculino,2,2.48
6,57,47,Masculino,1,1.94
7,58,56,Masculino,1,2.12
8,59,25,Femenino,2,2.50
9,60,45,Masculino,2,2.90


# No chambeo mas, porque ya me dio pereza ~cmmz 


**Extra:Posible muestra de asientos usados en la sala**

In [11]:
def crear_matriz():
    filas = ['A', 'B', 'C', 'D', 'E','F','G']
    columnas = [1, 2, 3, 4, 5, 6, 7]

    matriz = []

    for fila in filas:
        nueva_fila = []
        for columna in columnas:
            # La disponibilidad se establece inicialmente en True para todas las celdas
            celda = {'fila': fila, 'columna': columna, 'disponibilidad': True}
            nueva_fila.append(celda)
        matriz.append(nueva_fila)

    return matriz

def imprimir_matriz(matriz):
     for fila in matriz:
        for celda in fila:
            # Seleccionar el código de color basado en la disponibilidad
            color = '\033[92m' if celda['disponibilidad'] else '\033[91m'
            
            # Imprimir la celda en formato 'FilaColumna' con el color correspondiente
            print(f"{color}{celda['fila']}{celda['columna']}\033[0m", end='\t')
        print()  # Saltar a una nueva línea después de imprimir una fila completa

# Crear la matriz
mi_matriz = crear_matriz()

# Imprimir la matriz
imprimir_matriz(mi_matriz)

A1	A2	A3	A4	A5	A6	A7	
B1	B2	B3	B4	B5	B6	B7	
C1	C2	C3	C4	C5	C6	C7	
D1	D2	D3	D4	D5	D6	D7	
E1	E2	E3	E4	E5	E6	E7	
F1	F2	F3	F4	F5	F6	F7	
G1	G2	G3	G4	G5	G6	G7	
